### Import Packages

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sqlean as sqlite3
from functools import partial
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

### Import Data

In [2]:
DATABASE_PATH = 'C:/University/6G7V0007_MSC_Project/Project/Data/joblistings_transformed_OG_05072024.db'
con = sqlite3.connect(DATABASE_PATH)

In [3]:
job = pd.read_sql('SELECT * FROM job', con)
company = pd.read_sql('SELECT * FROM company', con)
website = pd.read_sql('SELECT * FROM website', con)

C:\Users\walte\AppData\Local\Temp\ipykernel_12644\1361633401.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  job = pd.read_sql('SELECT * FROM job', con)
C:\Users\walte\AppData\Local\Temp\ipykernel_12644\1361633401.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  company = pd.read_sql('SELECT * FROM company', con)
C:\Users\walte\AppData\Local\Temp\ipykernel_12644\1361633401.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  website = pd.read_sql('SELECT * FROM website', con)


In [4]:
job.set_index('id', inplace=True)
company.set_index('id', inplace=True)
website.set_index('id', inplace=True)

In [5]:
test = job.merge(company, left_on='company_id', right_index=True)

In [6]:
test.rename(columns={'name':'company_name'}, inplace=True)

In [7]:
test.tail(10)

,website_id,company_id,title,location,pay,description,timestamp,company_name
id,,,,,,,,
1917,2,671,Product Manager,Bracknell,"£41,000 - £59,000 a year",Product Manager – Bracknell – Hybrid – Up to £...,2024-07-05,Step Ahead Recruitment Ltd
1920,2,672,Software Architect,"4 Phoenix Square, Colchester CO4","£52,000 - £65,000 a year","Nordson Test & Inspection, a leading provider ...",2024-07-05,Nordson
1921,2,673,Graduate Technology Consultant - PhD Level,Guildford,"£34,000 a year",This hugely successful and rapidly expanding c...,2024-07-05,Graduate Recruitment Bureau
1922,2,674,BI Analyst,Alderley Edge,NULL,CurrentBody is seeking a BI Analyst to join ou...,2024-07-05,CurrentBody
1923,2,675,Full Stack Developer,Remote,"£45,397.82 - £55,353.46 a year",Please visit our website before applying https...,2024-07-05,PureCode Software
1926,2,676,Network Engineer,4it Recruitment Ltd in Manchester,"Up to £55,000 a year",Network Engineer – Palo Alto/Cisco - £55k – Ma...,2024-07-05,4it Recruitment Ltd
1927,2,677,Senior Software Developer,prosperIS Recruitment Ltd in Monmouth,"£40,000 - £50,000 a year",*Are you a experienced Software Developer who'...,2024-07-05,prosperIS Recruitment Ltd
1928,2,678,Junior Systems Engineer (UK-based),Didcot,NULL,"Work location: United Kingdom, England, Didcot...",2024-07-05,D-Orbit
1929,2,679,Engineer (Test & Validation),Telford TF7,"£26,000 - £36,000 a year",ENGINEERING TECHNICIAN: TEST & VALIDATION JOB ...,2024-07-05,VA Technology Ltd.


In [8]:
test.loc[test['location'].str.contains('Morley')]

,website_id,company_id,title,location,pay,description,timestamp,company_name
id,,,,,,,,
1916,2,670,Manual Test Lead,Morley LS27,"From £57,500 a year",At Evri we understand that searching for your ...,2024-07-05,Evri


### Clean location

In [9]:
# Remove mentions of hybrid in location
test['location'].replace(r'\(Hybrid\)', '', regex=True, inplace=True)

In [10]:
# Remove first line of address
test['location'].replace(r'[0-9]*[a-zA-Z\s]*\,', '', regex=True, inplace=True)

In [11]:
# Strip whitespace
test['location'] = test['location'].str.strip()

In [12]:
# Map multiple locations to separate category
test['location'].replace(r'.*\sand\s.*', 'Multiple Locations', regex=True, inplace=True)
test['location'].replace(r'.*\s[&+]\s.*', 'Multiple Locations', regex=True, inplace=True)
test['location'].replace('Multiple UK Locations', 'Multiple Locations', inplace=True)
test['location'].replace('Multiple Worldwide Locations', 'Multiple Locations', inplace=True)

In [13]:
test['location'].unique()

array(['Cirencester (Gloucestershire)', 'London', 'Manchester',
       'Multiple Locations', 'Bristol', 'Cambridge',
       'Geneva (Switzerland)', 'Leeds', 'Warsaw (Poland)',
       'Brixworth (Northamptonshire)', 'Glasgow',
       'Broad Oak (Dorchester)', 'Barrow-in-Furness (Cumbria)',
       'Filton (Bristol)', 'Broad Oak (Portsmouth)', 'Frimley (Surrey)',
       'Weymouth (Dorset)', 'Barrow (Cumbria)', 'Brough (Hull)',
       'Prestwick (South Ayrshire)', 'Barrow-In-Furness', 'Brough',
       'Chippenham (Wiltshire)', 'Pontyclun (South Wales)',
       'Leatherhead (Surrey)', 'Pontyclun (Wales)',
       'Redditch (Worcestershire)', 'Trafford Park (Manchester)',
       'Stafford (West Midlands)', 'Cheltenham (Gloucestershire)',
       'Didsbury (Manchester)', 'Solihull', 'Brighton', 'Remote',
       'Malvern Hills (Worcestershire)', 'Chiswick (London)',
       'Chiswick Park', 'Luton (Bedfordshire)', 'Edinburgh',
       'Basildon (Essex)', 'Southampton', 'Leicester', 'Oxford',
     

In [14]:
test['location'].value_counts(sort=True).head(25)

location
London                         245
Remote                          86
Multiple Locations              84
Geneva (Switzerland)            36
Leeds                           32
Cambridge                       31
Manchester                      25
London EC4N                     20
Bristol                         15
Bridgend                        14
Edinburgh                       13
Budapest                        12
Birmingham                      10
Oxford                           9
Douglas                          8
Southampton                      7
Glasgow                          7
Reading                          7
Bollington (Cheshire)            6
Barrow-in-Furness (Cumbria)      6
London SE1                       6
Liverpool                        6
Swindon                          5
Newcastle upon Tyne              5
Stafford (West Midlands)         5
Name: count, dtype: int64

In [22]:
# Remove entries where location is in Switzerland, Japan, Poland, Cbina or Budapest 
test = test.loc[~test['location'].str.contains('Switzerland|Japan|Budapest|Poland|China')]

In [23]:
test['location'].value_counts(sort=True).head(25)

location
London                          245
Remote                           86
Multiple Locations               84
Leeds                            32
Cambridge                        31
Manchester                       25
London EC4N                      20
Bristol                          15
Bridgend                         14
Edinburgh                        13
Birmingham                       10
Oxford                            9
Douglas                           8
Reading                           7
Southampton                       7
Glasgow                           7
Barrow-in-Furness (Cumbria)       6
Bollington (Cheshire)             6
Liverpool                         6
London SE1                        6
Stoke-on-Trent                    5
Stafford (West Midlands)          5
Newcastle upon Tyne               5
Manchester M4                     5
Brixworth (Northamptonshire)      5
Name: count, dtype: int64

In [24]:
# Define geocoder objects
geolocator = Nominatim(user_agent='msc_project')
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

In [27]:
# Get full location with geocoder
test['full_loc'] = test['location'].apply(partial(geocode, language='en', addressdetails=True))
test.loc[test['location'].isin(['Remote', 'Multiple Locations']), 'full_loc'] = None

In [37]:
# Get latitude and longitude
test['lat'] = test['full_loc'].apply(lambda x: x.latitude if x else None)
test['long'] = test['full_loc'].apply(lambda x: x.longitude if x else None)

In [38]:
# Get raw location data
test['raw_loc'] = test['full_loc'].apply(lambda x: x.raw if x else None)

In [39]:
test[['location', 'full_loc', 'lat', 'long', 'raw_loc']].tail(30)

,location,full_loc,lat,long,raw_loc
id,,,,,
1867,London EC3N,"(Royal Mail, Crosswall, Aldgate, City of Londo...",51.511505,-0.077226,"{'place_id': 243623237, 'licence': 'Data © Ope..."
1872,London,"(London, Greater London, England, United Kingd...",51.507446,-0.127765,"{'place_id': 243408926, 'licence': 'Data © Ope..."
1873,Remote,None,NaN,NaN,None
1874,Nottingham NG8,"(NG8 292D, Nuthall Gardens, Whitemoor, Notting...",52.971066,-1.187053,"{'place_id': 243353141, 'licence': 'Data © Ope..."
1876,United Kingdom,"(United Kingdom, (54.7023545, -3.2765753))",54.702354,-3.276575,"{'place_id': 239127902, 'licence': 'Data © Ope..."
1881,Welwyn Garden City,"(Welwyn Garden City, Welwyn Hatfield, Hertford...",51.803108,-0.206887,"{'place_id': 244236001, 'licence': 'Data © Ope..."
1882,79- London SE1,None,NaN,NaN,None
1884,Harrow,"(London Borough of Harrow, London, Greater Lon...",51.596827,-0.337305,"{'place_id': 242260157, 'licence': 'Data © Ope..."
1885,Croydon CR0,"(Royal Mail, Commerce Way, Waddon, London Boro...",51.374417,-0.119376,"{'place_id': 244907691, 'licence': 'Data © Ope..."


In [40]:
test['raw_loc'].loc[1920]

{'place_id': 389807568,
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright',
 'osm_type': 'node',
 'osm_id': 11925916885,
 'lat': '51.913426',
 'lon': '0.8911755',
 'class': 'amenity',
 'type': 'post_box',
 'place_rank': 30,
 'importance': 6.33745326924089e-05,
 'addresstype': 'amenity',
 'name': 'CO4 320D',
 'display_name': 'CO4 320D, Nayland Road, Myland, Mile End, Colchester, Essex, England, CO4 5EN, United Kingdom',
 'address': {'amenity': 'CO4 320D',
  'road': 'Nayland Road',
  'suburb': 'Myland',
  'city': 'Colchester',
  'municipality': 'Essex',
  'ISO3166-2-lvl6': 'GB-ESS',
  'county': 'Essex',
  'state': 'England',
  'ISO3166-2-lvl4': 'GB-ENG',
  'postcode': 'CO4 5EN',
  'country': 'United Kingdom',
  'country_code': 'gb'},
 'boundingbox': ['51.9133760', '51.9134760', '0.8911255', '0.8912255']}

In [41]:
def get_settlement(raw_loc):
    try:
        address = raw_loc['address']
    except:
        return None
    try:
        return address['town']
    except:
        try:
            return address['village']
        except:
            try:
                return address['city']
            except:
                return None

In [42]:
def get_state(raw_loc):
    try:
        address = raw_loc['address']
        return address['state']
    except:
        return None

In [43]:
def get_country(raw_loc):
    try:
        address = raw_loc['address']
        return address['country_code']
    except:
        return None

In [44]:
test['settlement'] = test['raw_loc'].apply(get_settlement)
test['state'] = test['raw_loc'].apply(get_state)
test['country'] = test['raw_loc'].apply(get_country)

In [45]:
test[['location', 'settlement']].tail(30)

,location,settlement
id,,
1867,London EC3N,City of London
1872,London,London
1873,Remote,None
1874,Nottingham NG8,Nottingham
1876,United Kingdom,None
1881,Welwyn Garden City,Welwyn Garden City
1882,79- London SE1,None
1884,Harrow,London
1885,Croydon CR0,London


In [46]:
test['settlement'].unique()

array(['Cirencester', 'London', 'Manchester', None, 'Bristol',
       'Cambridge', 'Leeds', 'Brixworth', 'Glasgow', 'Barrow-in-Furness',
       'Filton', 'Portsmouth', 'Frimley', 'Weymouth', 'Brantingham',
       'Prestwick', 'Stank', 'Brough', 'Chippenham', 'Leatherhead',
       'Pontyclun', 'Redditch', 'Trafford', 'Stafford', 'Cheltenham',
       'Metropolitan Borough of Solihull', 'Brighton', 'Malvern Hills',
       'City of Edinburgh', 'Basildon', 'Southampton', 'Leicester',
       'Oxford', 'Bridgend', 'Burton-on-Trent', 'Derby', 'Shipley',
       'Poole', 'Newport', 'Newcastle upon Tyne', 'Norwich',
       'Hemel Hempstead', 'Bath', 'Reading', 'Milton Keynes', 'Dublin',
       'Eastbourne', 'Wolverhampton', 'Fareham', 'Nerston Village',
       'Thame', 'Birmingham', 'Gravesend', 'Aylesbury', 'Clermont',
       'Liverpool', 'Stratford-upon-Avon', 'Plymouth', 'Rosyth',
       'Stowmarket', 'Cardiff', 'Telford', 'Worthing', 'Exeter', 'Stroud',
       'Belfast', 'Bollington', 'Andove

In [47]:
test['settlement'].value_counts(sort=True).head(20)

settlement
London                 312
Cambridge               38
Manchester              36
Leeds                   36
City of London          31
Bristol                 17
City of Edinburgh       16
Bridgend                15
Oxford                  12
Birmingham              11
Glasgow                 10
Newcastle upon Tyne     10
Liverpool                9
Barrow-in-Furness        8
Derby                    8
Belfast                  8
Reading                  7
Southampton              7
Bollington               6
Stoke-on-Trent           5
Name: count, dtype: int64

In [48]:
test.loc[test['settlement'] == 'Stank']

,website_id,company_id,title,location,pay,description,timestamp,company_name,full_loc,lat,long,raw_loc,settlement,state,country
id,,,,,,,,,,,,,,,
431,2,14,Graduate Software Engineer,Barrow-In-Furness,"£34,000 a year",Description Job Title Graduate Submarines Soft...,2024-06-28,BAE Systems,"(Barrow-in-Furness, Stank, Westmorland and Fur...",54.12888,-3.226901,"{'place_id': 239373006, 'licence': 'Data © Ope...",Stank,England,gb


In [49]:
test['state'].value_counts(dropna=False)

state
England                   837
None                      213
Scotland                   41
Wales                      27
Northern Ireland            9
Illinois                    9
Nouvelle-Aquitaine          2
Massachusetts               1
Nebraska                    1
Central Albania             1
Alabama                     1
Maine                       1
Mecklenburg-Vorpommern      1
Nova Scotia                 1
Hong Kong                   1
KwaZulu-Natal               1
Rhode Island                1
Masovian Voivodeship        1
Virginia                    1
Name: count, dtype: int64

In [50]:
test['state'].unique()

array(['England', None, 'Masovian Voivodeship', 'Scotland', 'Wales',
       'Rhode Island', 'KwaZulu-Natal', 'Hong Kong', 'Northern Ireland',
       'Mecklenburg-Vorpommern', 'Nova Scotia', 'Illinois',
       'Massachusetts', 'Nouvelle-Aquitaine', 'Maine', 'Alabama',
       'Central Albania', 'Nebraska', 'Virginia'], dtype=object)

In [51]:
test['country'].unique()

array(['gb', None, 'pl', 'us', 'ie', 'za', 'cn', 'de', 'ca', 'fr', 'al'],
      dtype=object)

In [55]:
test.loc[test['country'] == 'fr']

,website_id,company_id,title,location,pay,description,timestamp,company_name,full_loc,lat,long,raw_loc,settlement,state,country
id,,,,,,,,,,,,,,,
446,2,189,Graduate Software Developer,Chard,"From £29,000 a year",We are seeking a Software Developer Graduate t...,2024-06-28,Numatic International Limited,"(Chard, Aubusson, Creuse, Nouvelle-Aquitaine, ...",45.94409,2.47593,"{'place_id': 105785849, 'licence': 'Data © Ope...",Chard,Nouvelle-Aquitaine,fr
1120,2,189,Firmware Graduate (R&D),Chard,"£29,000 a year",About Us We are looking for people who enjoy t...,2024-06-28,Numatic International Limited,"(Chard, Aubusson, Creuse, Nouvelle-Aquitaine, ...",45.94409,2.47593,"{'place_id': 105785849, 'licence': 'Data © Ope...",Chard,Nouvelle-Aquitaine,fr


In [56]:
test['settlement'].value_counts()

settlement
London            312
Cambridge          38
Manchester         36
Leeds              36
City of London     31
                 ... 
Newmarket           1
Belper CP           1
Carrickfergus       1
Cockermouth         1
Chiswick            1
Name: count, Length: 199, dtype: int64

### Plot on map

In [57]:
import plotly.express as px
import plotly.io as pio

In [58]:
pio.renderers.default = 'notebook_connected'

In [59]:
geodf = test.groupby('settlement').agg({'lat':'mean', 'long':'mean', 'settlement':'count'})

In [60]:
geodf.rename(columns={'settlement':'num_jobs'}, inplace=True)

In [61]:
fig = px.scatter_mapbox(geodf, lat='lat', lon='long', size='num_jobs', size_max=25, hover_name=geodf.index, zoom=4, mapbox_style='open-street-map')
fig.show()

In [62]:
fig = px.density_mapbox(geodf, lat='lat', lon='long', z='num_jobs', opacity=0.9, radius=30, center=dict(lat=52, lon=0), zoom=4, mapbox_style='open-street-map')
fig.show()